In [1]:
import numpy as np
import numpy.random as np_rand
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import math

# generating matrix with random black points

def fill_matrix(n, d):
    p = np.zeros((n,n), dtype = int)
    k = int(n*n*d)
    for i in range(0,k):
        x, y = np_rand.randint(0,n,(2))
        while p[x,y] == 1:
            x, y = np_rand.randint(0,n,(2))
        p[x,y] = 1
    return p

# energy

def get_close_neighbours(x, y, n):
    l = []
    for x2 in range(x-1, x+2):
        for y2 in range(y-1, y+2):
            if (-1 < x < n 
                and -1 < y < n 
                and (x != x2 or y != y2) 
                and (0 <= x2 < n) 
                and (0 <= y2 < n)):
                l.append((x2, y2))
    return l

def i_energy(points, local_energy):
    n = len(points)
    sum = 0
    for x in range(0,n):
        for y in range(0,n):
            sum += local_energy(x, y, points)
    return sum

def i_energy_n(points_old, points_new, energy_old, coords1, coords2, local_energy):
    n = len(points_old)
    sum = 0
    energy_new = energy_old
    x1,y1 = coords1
    x2,y2 = coords2
    nlist1 = get_close_neighbours(x1, y1, n)
    nlist2 = get_close_neighbours(x2, y2, n)
    sum1 = local_energy(x1, y1, points_old)
    sum2 = local_energy(x2, y2, points_old)
    for xn, yn in nlist1:
        sum1 += local_energy(xn, yn, points_old)
    for xn, yn in nlist2:
        sum2 += local_energy(xn, yn, points_old)
    energy_new -= sum1
    energy_new -= sum2
    sum1 = local_energy(x1, y1, points_new)
    sum2 = local_energy(x2, y2, points_new)
    for xn, yn in nlist1:
        sum1 += local_energy(xn, yn, points_new)
    for xn, yn in nlist2:
        sum2 += local_energy(xn, yn, points_new)
    energy_new += sum1
    energy_new += sum2
    return energy_new

def local_close_energy(x, y, points):
    nlist = get_close_neighbours(x,y,n)
    local_sum = points[x,y]
    for i, j in nlist:
        local_sum += points[i,j]
    if local_sum >= 4:
        local_sum += 1
    elif local_sum >= 6:
        local_sum += 2
    elif local_sum >= 8:
        local_sum += 3
    return local_sum

def local_attract_energy(x, y, points):
    nlist = get_close_neighbours(x,y,n)
    local_sum = 0
    for i, j in nlist:
        local_sum += points[i,j]
    if local_sum > 4:
        return local_sum
    else:
        return -local_sum

def local_diag_energy(x,y,points):
    nlist = get_close_neighbours(x,y,n)
    local_sum = 0
    if x % 2 == y % 2:
        local_sum += points[x,y]
        for i, j in nlist:
            if i % 2 == j % 2:
                local_sum += points[i,j]
    else:
        if points[x,y] == 0:
            local_sum += 1
            for i, j in nlist:
                if i % 2 != j % 2 and points[i,j] == 0:
                    local_sum += 1
    return local_sum


def local_even_energy(x,y,points):
    nlist = get_close_neighbours(x,y,n)
    local_sum = 0
    if x % 2 == 0 and y % 2 == 0:
        local_sum += points[x,y]
    for i, j in nlist:
        if i % 2 == 0 and j % 2 == 0:
            local_sum += points[i,j]
    return local_sum


def matrix_center(n):
    a = []
    if n % 2 == 0:
        c1 = n // 2
        a += [(c1,c1),(c1,c1-1),(c1-1,c1),(c1-1,c1-1)]
    else:
        a += [(c1,c1)]
    return a

def taxi_dist(x1, y1, x2, y2):
    return max(abs(x1 - x2), abs(y1 - y2))

def local_taxi_energy(x,y,points,c):
    sum = 0
    if (points[x,y] > 0):                
        dist_list = []
        for i, j in c:
           dist_list.append(taxi_dist(x,y,i,j))
        sum += min(dist_list)
    return sum

def taxi_energy(points, local_energy):
    n = len(points)
    sum = 0
    c = matrix_center(n)
    for x in range(0,n):
        for y in range(0,n):
            sum += local_energy(x,y,points,c)
    return sum

def taxi_energy_n(points_old, points_new, energy_old, coords1, coords2, local_energy):
    n = len(points_old)
    sum = 0
    c = matrix_center(n)
    energy_new = energy_old
    x1,y1 = coords1
    x2,y2 = coords2
    energy_new -= local_energy(x1, y1, points_old, c)
    energy_new -= local_energy(x2, y2, points_old, c)
    energy_new += local_energy(x1, y1, points_new, c)
    energy_new += local_energy(x2, y2, points_new, c)
    return energy_new

# schedule functions

def lin_schedule(t_0, k, n):
    a = np.linspace(0,t_0,n)
    return -a[k]+t_0

def exp_schedule(t_0, k, n):
    return t_0*(0.85**k)

def quad_schedule(t_0, k, n):
    return t_0/(1+2*k^2)

# swap functions

def arbit_swap(s, n):
    a = []
    coords = []
    for i in range(0,n):
        copy = s.copy()
        x1, y1 = np_rand.randint(0,len(s)-1,(2))
        x2, y2 = np_rand.randint(0,len(s)-1,(2))
        while (x1 == x2 and y1 == y2) or (s[x1,y1] == 0 and s[x2,y2] == 0):
            x1, y1 = np_rand.randint(0,len(s)-1,(2))
            x2, y2 = np_rand.randint(0,len(s)-1,(2))
        copy[x1,y1], copy[x2,y2] = copy[x2,y2], copy[x1,y1]
        a.append(copy)
        coords.append([(x1, y1), (x2, y2)])
    return a, coords

def close_swap(s, n):
    a = []
    coords = []
    for i in range(0,n):
        copy = s.copy()
        x1, y1 = np_rand.randint(0,len(s)-1,(2))
        neib = get_close_neighbours(x1, y1, len(s))
        neib_i = np_rand.randint(0,len(neib))
        x2, y2 = neib[neib_i]
        copy[x1,y1], copy[x2,y2] = copy[x2,y2], copy[x1,y1]
        a.append(copy)
        coords.append([(x1, y1), (x2, y2)])
    return a, coords

# probability

def p(e, e_new, t):
    if t < 1:
        t = 1
    return math.exp(-(e - e_new)/t)

# simulated annealing

def sim_an(s, k_max, t_0, t_1, p, schedule, swap, energy, energy_n, local_energy, neighbours_num):
    start_energy = energy(s, local_energy)
    for i in range(0, 5):
        for k in range(0, k_max):
            t = schedule(t_0, k, k_max)
            a, coords = swap(s, neighbours_num)
            for i in range(0,len(a)):
                new_energy = energy_n(s, a[i], start_energy, coords[i][0], coords[i][1], local_energy)
                if p(start_energy, new_energy, t) >= np_rand.random():
                    s = a[i]
                    start_energy = new_energy
            if math.isclose(t_1, t, rel_tol = 0.001):
                break
    return s


In [2]:
n = 128
d = 0.2
s = fill_matrix(n,d)
cmap = clr.ListedColormap(['w', 'k'])

In [3]:
s = np.loadtxt('array.txt')
s = s.astype(int)

In [9]:
plt.matshow(s, cmap = cmap)
plt.savefig('start.png')
plt.clf()

In [14]:
energy = taxi_energy
energy_n = taxi_energy_n
local_energy = local_taxi_energy

schedules = [lin_schedule, exp_schedule, quad_schedule]
swaps = [arbit_swap, close_swap]

energy_old = energy(s, local_energy)
print(energy_old)

name = 'taxi_'

for i in range(0,len(schedules)):
    for j in range(0,len(swaps)):
        s_new = sim_an(s, 1000, 3, 0.001, p, schedules[i], swaps[j], energy, energy_n, local_energy, 50)
        new_name = name + str(i) + str(j)
        plt.matshow(s_new, cmap = cmap)
        plt.savefig(new_name + '.png')
        plt.clf()
        filename = "results.txt"
        file = open(filename, "a")
        file.write(new_name + ' = ' + str(energy(s_new, local_energy)) + '\n')

137500
